In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
i = 30
y, I = np.genfromtxt(f'SEMSANS_Delft/{i}/det.dat', delimiter=' ', usecols=(0,1), unpack=True)
_, I_base = np.genfromtxt(f'SEMSANS_Delft_no_sample/{i}/det.dat', delimiter=' ', usecols=(0,1), unpack=True)
_, I_flat = np.genfromtxt(f'SEMSANS_Delft_no_sample_no_analyzer/{i}/det.dat', delimiter=' ', usecols=(0,1), unpack=True)
_, I_unpolarized = np.genfromtxt(f'SEMSANS_Delft_no_sample_unpolarized/{i}/det.dat', delimiter=' ', usecols=(0,1), unpack=True)


y*=1e-2

plt.plot(y*1e3, I,'-',  label='Sample, analyzer')
plt.plot(y*1e3, I_base, label='No sample, analyzer')
plt.plot(y*1e3, I_flat, label='No sample, no analyzer')
plt.plot(y*1e3, I_unpolarized, label='No sample, unpolarized')

plt.xlabel('y [mm]')
plt.ylabel('I')
plt.title('PSD intensity')
plt.grid(True)  # Add grid
plt.legend()
plt.show()

In [ ]:
from definitions import *

compute_By(6.65e-6)

In [ ]:
max_By = compute_By(3e-6)
round(max_By, 6)

In [ ]:
def indices_within_range(x, a, b):
    return np.where((x >= a) & (x <= b))[0]

# def I_fit(y, a, b, p_inv):
#     return (a + b * np.cos(2*np.pi * y * p_inv))

# def I_fit_2(y, a, b, p_inv, d):
#     return (a + b * np.cos(2*np.pi * y * p_inv) * np.exp(-d * y**2))


# def I_mod(y, I_0, A, p_0):
#     return I_0 * (1 + A * np.cos(2*np.pi * y / p_0))

# def I_mod_2(y, I_0, A, p_0,d):
#     return I_0 * (1 + A * np.cos(2*np.pi * y / p_0) * np.exp(-d * y**2))

def fit_pattern(y, I, I_unpol, By, plot=False):
    p_0_pred = compute_p_0(By)
    p_inv = 1/p_0_pred
    indices = indices_within_range(y, -0.0005, 0.0005)

    # The DC component can be expected to be the mean
    mean = np.mean(I[indices])
    # The amplitude will be roughly the max minus the mean
    amplitude = (np.max(I[indices]) - mean)/mean

    # bounds = ([0.7 * mean, 0.7 * amplitude, 0.8*p_inv,0.0], [1.3 * mean, 1.3 * amplitude, 1.2*p_inv,2*np.pi])
    guess =  [amplitude, p_inv]
    def I_fit(y,A, p_inv):
        return I_unpol[indices] * (1 + A * np.cos(2*np.pi * y * p_inv))
    popt, pcov = curve_fit(I_fit, y[indices], I[indices],p0=guess)
    diag = np.diagonal(pcov)

    A_0 = popt[0]
    dA_0 = diag[0]
    p_0_fit = 1/popt[1]
    dp_0_fit = diag[1] / popt[1]**2
    if plot:    
        I_fitted = I_fit(y[indices],*popt)
        I_guess = I_fit(y[indices],*[amplitude, p_inv])
        plt.plot(y*1e3, I, '.', label='Simulation data')
        plt.plot(y[indices]*1e3, I_fitted, label='Fit')
        # plt.plot(y[indices]*1e3, I_guess, label='Guess')
        plt.xlabel('y [mm]')
        plt.ylabel('I')
        plt.grid(True)
        plt.legend()
        plt.show()
    return [A_0,p_0_fit], [dA_0,dp_0_fit]

def V_direct(y, I, min_y, max_y):
    indices = indices_within_range(y, min_y, max_y)
    I_max = np.max(I[indices])
    I_min = np.min(I[indices])
    print(I_max, I_min)
    V = (I_max - I_min) / (I_max + I_min)
    print(V)
    return V

In [ ]:
import re

def extract_By(file):
    with open(file) as f:
        text = f.read()
        match = re.search(r'# Param: By=([0-9.]+)', text)
        assert(match)
        return float(match.group(1))

N = 31
R = np.zeros((N))
dR = np.zeros((N))
z = np.zeros((N))
for i in range(0,N):
    By = extract_By(f"SEMSANS_Delft/{i}/det.dat")
    print(By, compute_p_0(By))
    y, I = np.genfromtxt(f"SEMSANS_Delft/{i}/det.dat", delimiter=' ', usecols=(0,1), unpack=True)
    _, I_base = np.genfromtxt(f"SEMSANS_Delft_no_sample/{i}/det.dat", delimiter=' ', usecols=(0,1), unpack=True)
    _, I_unpol = np.genfromtxt(f"SEMSANS_Delft_no_sample_unpolarized/{i}/det.dat", delimiter=' ', usecols=(0,1), unpack=True)
    y *= 1e-2
    params, dparams = fit_pattern(y, I, I_unpol, By, False)
    params_base, dparams_base = fit_pattern(y, I_base, I_unpol, By, False)
    A_z = params[0]
    A_0 = params_base[0]
    # V_z = V_direct(y,I, -0.004, 0.004) 
    # V_0 = V_direct(y,I_base, -0.004, 0.004) 
    z[i] = compute_z(By)
    R[i] = A_z/A_0
    dR[i] = 0.0  
combined_data = np.hstack((z.reshape(-1, 1), R.reshape(-1, 1)))
np.savetxt('simulation-fit.txt', combined_data, delimiter=',')
print("Data saved successfully.")
plt.plot(z * 1e6,R,'.')
plt.xlabel(r'z [$\mu m$]')
plt.ylabel(r'$\frac{A}{A_0}$')
plt.grid(True)